In [9]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import imodelsx.llm
from transformers import AutoModel, AutoTokenizer, pipeline
from tqdm import tqdm
from os.path import join
import config
import numpy as np
import joblib
fit_encoding = __import__('01_fit_encoding')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
sparse_feats_dir = join(config.repo_dir, 'sparse_feats')

In [6]:
!ls {sparse_feats_dir}

'v3_boostexamples_(0, -3, 15).joblib'


In [8]:
alphas_enet, coefs_enet = joblib.load(
    join(sparse_feats_dir, 'v3_boostexamples_(0, -3, 15).joblib'))

In [12]:
for i in range(len(alphas_enet)):
    coef_enet = coefs_enet[:, :, i]
    coef_nonzero = np.any(np.abs(coef_enet) > 0, axis=0)
    print(f'{i=} alpha={alphas_enet[i]:.2e}, n_nonzero={coef_nonzero.sum()}')

i=0 alpha=1.00e+00, n_nonzero=0
i=1 alpha=6.11e-01, n_nonzero=0
i=2 alpha=3.73e-01, n_nonzero=1
i=3 alpha=2.28e-01, n_nonzero=29
i=4 alpha=1.39e-01, n_nonzero=102
i=5 alpha=8.48e-02, n_nonzero=264
i=6 alpha=5.18e-02, n_nonzero=473
i=7 alpha=3.16e-02, n_nonzero=638
i=8 alpha=1.93e-02, n_nonzero=669
i=9 alpha=1.18e-02, n_nonzero=672
i=10 alpha=7.20e-03, n_nonzero=674
i=11 alpha=4.39e-03, n_nonzero=674
i=12 alpha=2.68e-03, n_nonzero=674
i=13 alpha=1.64e-03, n_nonzero=674
i=14 alpha=1.00e-03, n_nonzero=674
